In [1]:
import MySQLdb as mdb
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import timezone

In [2]:
# Create database connection
db_host = 'localhost'
db_user = 'crypto_user'
db_pass = 'Blue0811'
db_name = 'cryptocurrencies_master'
db_connection = mdb.connect(host=db_host, user=db_user,
                            passwd=db_pass, db=db_name)

In [3]:
# Extract cryptocurrency trading pairs from price_1min table
with db_connection.cursor() as cur:
    cur.execute('SELECT DISTINCT id FROM price_1min;')
    crypto_pairs = list(cur.fetchall())
print('Crypto-Pairs from 1-minute Data Table:')
for pair in crypto_pairs:
    print(pair[0])

Crypto-Pairs from 1-minute Data Table:
BCH-USD
BTC-USD
ETH-USD
LTC-USD


In [4]:
# Define function to transform unix timestamps to trading day indicies
def stamp_to_trading_day(timestamp):
    date = datetime.fromtimestamp(timestamp).astimezone(timezone('UTC'))
    minute = date.hour*60 + date.minute
    return (date.month, date.day, minute)

In [5]:
# Obtain volume data for each currency fom database
unix_start = 1572566460
unix_end = 1585008000
crypto_data = {}

for pair in crypto_pairs:
    pair = pair[0]
    with db_connection.cursor() as cur:
        execution_str = 'SELECT price_date, volume \nFROM price_1min \nWHERE id = "{}" AND '.format(pair)
        execution_str += '(price_date >= {} AND price_date < {})\n'.format(unix_start, unix_end)
        execution_str += 'ORDER BY price_date;'
        print('query:\n{}\n'.format(execution_str))
        cur.execute(execution_str)
        data = list(cur.fetchall())
    crypto_data[pair] = pd.DataFrame(data, columns=('price_date', 'volume'))
    crypto_data[pair]['price_date'] = crypto_data[pair]['price_date'].apply(stamp_to_trading_day)
    crypto_data[pair] = crypto_data[pair].set_index('price_date')

query:
SELECT price_date, volume 
FROM price_1min 
WHERE id = "BCH-USD" AND (price_date >= 1572566460 AND price_date < 1585008000)
ORDER BY price_date;

query:
SELECT price_date, volume 
FROM price_1min 
WHERE id = "BTC-USD" AND (price_date >= 1572566460 AND price_date < 1585008000)
ORDER BY price_date;

query:
SELECT price_date, volume 
FROM price_1min 
WHERE id = "ETH-USD" AND (price_date >= 1572566460 AND price_date < 1585008000)
ORDER BY price_date;

query:
SELECT price_date, volume 
FROM price_1min 
WHERE id = "LTC-USD" AND (price_date >= 1572566460 AND price_date < 1585008000)
ORDER BY price_date;



In [6]:
# Define the trading day indicies
trading_days = []
for m, n in ((11, 30), (12, 31), (1, 31), (2, 29), (3, 23)):
    trading_days += [(m, i) for i in range(1, n + 1)]
    
volume_cryptos = pd.DataFrame(columns=trading_days, index=range(0, 1440))

In [7]:
# Define function to compute the average volume for each trading day and minute 
def compute_vol_avg(trading_stamp, crypto_data):
    total_vol = 0
    for pair in crypto_data.keys():
        if crypto_data[pair].index.contains(trading_stamp):
            total_vol += crypto_data[pair]['volume'][trading_stamp]
        else:
            total_vol += 0
    return total_vol / len(crypto_data.keys())

In [8]:
# Compute volume data for the volume_cryptos table
for day in volume_cryptos.columns:
    for minute in volume_cryptos.index:
        trading_stamp = day + (minute,)
        volume_cryptos[day][minute] = compute_vol_avg(trading_stamp, crypto_data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  """


In [9]:
print('Data Shape: {}'.format(volume_cryptos.shape))
volume_cryptos.head()

Data Shape: (1440, 144)


,"(11, 1)","(11, 2)","(11, 3)","(11, 4)","(11, 5)","(11, 6)","(11, 7)","(11, 8)","(11, 9)","(11, 10)",...,"(3, 14)","(3, 15)","(3, 16)","(3, 17)","(3, 18)","(3, 19)","(3, 20)","(3, 21)","(3, 22)","(3, 23)"
0,0,9.52448,27.7077,28.2831,54.9851,78.7985,24.0819,33.2921,63.0435,6.37282,...,152.932,94.3192,75.45,1447.1,1165.97,300.085,700.473,51.8646,147.664,297.986
1,43.6276,213.665,61.8671,21.0084,21.4793,18.5754,77.7306,12.5094,72.7155,6.57368,...,0,53.7927,302.534,597.365,227.328,2080.33,200.396,108.703,197.378,6.78818
2,11.6722,40.4598,33.2271,8.03392,10.453,11.5149,203.406,4.09453,2.22694,4.11494,...,234.486,203.519,1660.25,157.184,250.954,276.849,146.67,332.711,100.294,3743.53
3,31.1001,5.30367,21.1073,1.75745,168.712,21.6921,13.9024,54.2846,1.26975,45.7626,...,142.891,647.077,553.666,188.07,124.122,368.93,279.159,778.891,17.6822,3288.48
4,7.15688,3.05717,79.5344,7.33955,24.5172,5.23118,13.1275,12.9724,88.4375,11.0792,...,375.477,118.862,182.785,311.361,144.336,172.522,155.211,192.128,51.8341,1558.43


In [10]:
# Compute volume data for the volume_cryptos table
volume_days = pd.DataFrame(columns=crypto_data.keys(), index=range(1440))
minutes = range(24*60)
trading_minutes = {minute: [day + (minute,) for day in trading_days] for minute in minutes}
for pair in crypto_data.keys():
    for minute in minutes:
        volume_days[pair][minute] = crypto_data[pair]['volume'][trading_minutes[minute]].mean()

/home/ryan/.local/lib/python3.6/site-packages/pandas/core/series.py:1152: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [11]:
print('Data Shape: {}'.format(volume_days.shape))
volume_days.head()

Data Shape: (1440, 4)


,BCH-USD,BTC-USD,ETH-USD,LTC-USD
0,63.8525,29.7433,240.679,360.408
1,53.2267,24.1464,187.44,271.83
2,53.3374,23.3499,225.264,292.39
3,45.0592,21.8377,193.611,230.347
4,39.4752,15.0297,149.243,163.77


In [12]:
# Write volume data tables to pickle files
volume_cryptos.to_pickle('./volume_cryptos.pkl')
volume_days.to_pickle('./volume_days.pkl')